Data Analysis for cycle wands project
Code will use OS NGD API to obtain data on cycle lanes in the UK
Analysis will be conducted to split cycle lanes by type and location (London Boroughs)

In [ ]:
%pip install -r requirements.txt

In [4]:
import requests
import json

# API features
api_key = "iKiHPsM65ooKNn5q48AgTlgJbkpmen6h"
collection_id = "trn-ntwk-cyclelane-1"
os_ngd_url = f"https://api.os.uk/features/ngd/ofa/v1/collections/{collection_id}/items"
# Parameters
request_parameters = {
    "key": api_key,
    "bbox": "-0.5202136724,51.3257416338,0.2584423334,51.6941179174",
    "limit": 100
}
os_response = requests.get(url = os_ngd_url, headers={'key': api_key}, params=request_parameters, timeout=40).json()

print("Number returned:", os_response.get("numberReturned"))
print("Number matched:", os_response.get("numberMatched"))


Number returned: 5
Number matched: None


In [7]:
import requests
import json
import geopandas as gpd
import folium
from shapely.geometry import shape

# --- API setup ---
api_key = "iKiHPsM65ooKNn5q48AgTlgJbkpmen6h"
collection_id = "trn-ntwk-cyclelane-1"
os_ngd_url = f"https://api.os.uk/features/ngd/ofa/v1/collections/{collection_id}/items"

# --- Request parameters (London bbox) ---
request_parameters = {
    "bbox": "-0.5202136724,51.3257416338,0.2584423334,51.6941179174",
    "limit": 100,
    "key": api_key
}

# --- Fetch data ---
os_response = requests.get(url=os_ngd_url, headers={'key': api_key}, params=request_parameters, timeout=40).json()

print("Number returned:", os_response.get("numberReturned"))
print("Number matched:", os_response.get("numberMatched"))

# --- Extract features ---
features = os_response.get("features", [])

# --- Convert to GeoDataFrame ---
geoms = []
for f in features:
    geom = f.get("geometry")
    if geom:
        geoms.append(shape(geom))

gdf = gpd.GeoDataFrame(geometry=geoms, crs="EPSG:4326")
print(f"Cycle lanes plotted: {len(gdf)}")

# --- Make a simple Folium map ---
if not gdf.empty:
    # center map on London
    m = folium.Map(location=[51.5074, -0.1278], zoom_start=10, tiles="CartoDB positron")

    folium.GeoJson(
        gdf.to_json(),
        name="OS NGD Cycle Lanes",
        style_function=lambda x: {"color": "green", "weight": 3, "opacity": 0.8}
    ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)
else:
    print("No geometries found.")


Number returned: 5
Number matched: None
Cycle lanes plotted: 5
